### Links
- https://dadosabertos.bcb.gov.br/dataset/expectativas-mercado/resource/d420a704-75a7-4f45-8f4b-0fca813c70f0
- https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/documentacao

In [ ]:
import pandas as pd
from IPython.display import display


# ------------------------------------------------------------------
parameters_total = '$format=text/csv&select=Indicador,IndicadorDetalhe,Data,DataReferencia,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo'
parameters_filter = '$top=10000&$format=text/csv&$orderby=Data%20desc&select=Indicador,IndicadorDetalhe,Data,DataReferencia,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo'


def getFocusData(parameters, filename):
    url = ('https://olinda.bcb.gov.br/olinda/servico/'
           'Expectativas/versao/v1/odata/ExpectativasMercadoAnuais?'+parameters
           )

    dffocus = pd.read_csv(url, decimal=",")
    dffocus.to_csv(filename, index=False)

    return dffocus


# ------------------------------------------------------------------
def removeFocusLastMonth(dffocus):
    dffocus_lessmonth = dffocus.copy()

    dffocus_tmp = dffocus.copy()
    dffocus_tmp['Data'] = pd.to_datetime(
        dffocus_tmp['Data'], format='%y-%m-%d', infer_datetime_format=True)

    dfcolumndata = dffocus_tmp['Data']
    dffocus_lessmonth.drop(
        dfcolumndata[dffocus_tmp['Data'] >= '2022-03-01'].index, inplace=True, axis=0)

    return(dffocus_lessmonth)


# ------------------------------------------------------------------
def addFocusColumns(dftotalfocus_lessmonth, dffocus_filter, filename):
    dffocus_added = dftotalfocus_lessmonth.append(
        dffocus_filter, ignore_index=True)
    dffocus_added = dffocus_added.drop_duplicates()
    dffocus_added.to_csv(filename, index=False)


# compares final result to verify data integrity
# ------------------------------------------------------------------
def compareFocusData(filename1, filename2, filename3, filename4):
    df_completo = pd.read_csv(filename1)
    df_completo = df_completo.sort_values(
        by=['Indicador', 'Data'], ignore_index=True)
    # save to disk for binary comparing
    df_completo.to_csv(filename2, index=False)

    df_somado_semdup = pd.read_csv(filename3)
    df_somado_semdup = df_somado_semdup.sort_values(
        by=['Indicador', 'Data'], ignore_index=True)
    # save to disk for binary comparing
    df_somado_semdup.to_csv(filename4, index=False)

    # compares dataframes
    df_completo.equals(df_somado_semdup)


# MAIN
# ------------------------------------------------------------------
dftotalfocus = getFocusData(parameters_total, 'ftotal.csv')
dftotalfocus_lessmonth = removeFocusLastMonth(dftotalfocus)
dffocus_filter = getFocusData(parameters_filter, 'ffilter.csv')
addFocusColumns(dftotalfocus_lessmonth, dffocus_filter, 'ffinal.csv')
compareFocusData('ftotal.csv', 'ftotaltocompare.csv',
                 'ffinal.csv', 'ffinaltocompare.csv')
